In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

try:
    import text_recognizer
except ImportError:
    import sys
    sys.path.append('..')

from text_recognizer.datasets import IamDataset
from text_recognizer.datasets.iam_paragraphs_dataset import IAMParagraphsDataset
from text_recognizer.models.line_detector_model import LineDetectorModel
from text_recognizer import util

In [7]:
paragraphs_dataset = IAMParagraphsDataset()
paragraphs_dataset.load_or_generate_data()
print(paragraphs_dataset)

Loading IAM paragraph crops and ground truth from image files...
IAM Paragraphs Dataset
Num classes: 3
Train: (1229, 256, 256) (1229, 256, 256, 3)
Test: (308, 256, 256) (308, 256, 256, 3)



In [ ]:
line_detector_model = LineDetectorModel()
line_detector_model.load_weights()

In [ ]:
for ind in range(5):
    image = paragraphs_dataset.x_test[ind]
    gt = paragraphs_dataset.y_test[ind].argmax(-1)
    pred = line_detector_model.predict_on_image(image).argmax(-1)

    fig = plt.figure(figsize=(15,15))
    ax1 = fig.add_subplot(131)
    ax1.matshow(image, cmap='gray')
    ax2 = fig.add_subplot(132)
    ax2.matshow(gt, cmap='gray')
    ax3 = fig.add_subplot(133)
    ax3.matshow(gt, cmap = 'gray')